In [1]:
import sys
sys.path.append("/Users/nrahaman/Python/info-bazaar")
sys.path.append("/Users/nrahaman/Python/speedrun")

In [232]:
import pandas as pd
import seaborn as sns
from pathlib import Path
from itertools import product
from tqdm import tqdm

from bazaar.lem_utils import ask_for_guidance, clean_program_string, get_llm, get_closed_book_answer
from bazaar.py_utils import dump_dict

In [3]:
noinspect_path = Path("/Users/nrahaman/Downloads/likert_debate_eval_gpt-3.5-turbo_fupnoinspect-STAR-RemoteLlama-2-70b-chat-hf-STAR.csv")
normal_path = Path("/Users/nrahaman/Downloads/likert_debate_eval_gpt-3.5-turbo_fup-v3-STAR-RemoteLlama-2-70b-chat-hf-STAR.csv")

In [4]:
noinspect_df = pd.read_csv(noinspect_path)
normal_df = pd.read_csv(normal_path)

In [5]:
def get_unique_qs(df): 
    return df["question"].unique().tolist()

In [8]:
shared_qs = list(set(get_unique_qs(noinspect_df)).intersection(set(get_unique_qs(normal_df))))

In [9]:
shared_qs

['What are the potential consequences of using LLMs in high-stakes applications without proper interpretability?',
 'What is the primary focus of work on scaling laws in the context of LLMs?',
 "Can in-context learning occur when there's a distribution mismatch between prompts and pretraining data?",
 'What is the difference between LLaMA-Adapter and LLaMA-Adapter-V2?',
 'What is the purpose of cross-validation in machine learning?',
 'What is the purpose of model selection in machine learning?',
 'What is reinforcement learning?',
 'What is the difference between a BERT and a GPT?',
 'What is the purpose of pre-training in LLMs?',
 'How many days after its release did ChatGPT attract over a million users?',
 'What is the meaning of "emergent capabilities" in the context of large language models?',
 'What is the significance of the date November 30th, 2022, in the development of language models?',
 'How do biases and fairness issues arise in LLMs, and what can be done to mitigate them?

In [10]:
shared_qs[0]

'What are the potential consequences of using LLMs in high-stakes applications without proper interpretability?'

In [224]:
def compare_answers(question, answer_a, answer_b, return_full_answer=False):
    program_str = """
    {{#system~}}
    Michael and Bobby are two machine learning professors grading exams. 
    
    They have a question, and two answers to those questions from two students. In addition, they also have the gold standard answer. Their task is to determine which of the two answers does a better job at answering the question. 
    
    A good answer addresses the question directly with clarity and is easy to understand. It does not go off on tangents, but surfaces information relevant to the question. 
    
    You will simulate a heated debate between Michael and Bobby as they deliberate on this. At the end of the argument, they converge to a fair verdict. The verdict must not be a tie and it must be printed as: 
    
    VERDICT: Better answer from student <A or B>
    {{~/system}}
    
    {{#user~}}
    Question: {{question}}
    
    Answer from student A: {{answer_a}}
    
    Answer from student B: {{answer_b}}
    
    Gold standard answer: {{answer_gold}}
    {{~/user}}
    
    {{#assistant~}}
    {{gen "answer" max_tokens=1024 temperature=0.0}}
    {{~/assistant}}
    """
    program_str = clean_program_string(program_str)
    
    gold_answer = get_closed_book_answer(question, "gpt-4")
    
    program_output = ask_for_guidance(
        program_string=program_str, 
        llm=get_llm(model_name="gpt-4"),
        silent=True,
        inputs=dict(
            question=question,
            answer_a=answer_a,
            answer_b=answer_b,
            answer_gold=gold_answer,
        ),
        output_keys=["answer"],
    )
    
    answer = program_output["answer"]
    
    verdict_lines = [l for l in answer.splitlines() if l.startswith("VERDICT")]
    assert len(verdict_lines) == 1
    verdict_line = verdict_lines[0]
    if verdict_line[-1] in ["A", "B"]: 
        verdict = verdict_line[-1]
    else:
        verdict = verdict_line[-2]
    if return_full_answer:
        return verdict, answer, gold_answer
    else: 
        return verdict

In [82]:
def get_most_expensive_answer(df, question): 
    row = df[df["question"] == question].sort_values(by="credit_spent").iloc[-1]
    return row.answer, row.credit_spent

In [166]:
def get_answer_closest_to_price(df, price, question): 
    df = df[df["question"] == question]
    df["price_diff"] = df["credit_spent"].apply(lambda c: abs(c - price))
    row = df.sort_values(by="price_diff").iloc[0]
    return row.answer, row.credit_spent

In [201]:
def keep_at_budget(df, budget): 
    return df[df["budget"].round() == budget]


def keep_at_question(df, question): 
    return df[df["question"] == question]

In [186]:
noinspect_df["budget"].round().unique()

array([ 25.,  10.,  50., 100.,   5., 200.])

In [265]:
# def get_answer_pair(question, budget=None): 
#     noinspect_for_q = noinspect_df[noinspect_df["question"] == question]
#     normal_for_q = normal_df[normal_df["question"] == question]
#     # Keep answers with a budget
#     if budget is not None: 
#         noinspect_for_q = keep_at_budget(noinspect_for_q, budget)
#         normal_for_q = keep_at_budget(normal_for_q, budget)
#     # Get the most expensive noinspect answer
#     price_point = noinspect_for_q.sort_values(by="credit_spent").iloc[-1].credit_spent
#     noinspect_answer, noinspect_price = get_answer_closest_to_price(noinspect_df, price_point, question)
#     normal_answer, normal_price = get_answer_closest_to_price(normal_df, price_point, question)
#     return (noinspect_answer, noinspect_price), (normal_answer, normal_price)

In [266]:
def get_answer_pair(question, budget): 
    noinspect_row = keep_at_budget(keep_at_question(noinspect_df, question), budget).iloc[0]
    normal_row = keep_at_budget(keep_at_question(normal_df, question), budget).iloc[0]
    return (noinspect_row.answer, noinspect_row.credit_spent), (normal_row.answer, normal_row.credit_spent)

In [208]:
def get_answer_pair_for_budgets(df, question, budget_a, budget_b): 
    df = keep_at_question(df, question)
    df_a = keep_at_budget(df, budget_a)
    df_b = keep_at_budget(df, budget_b)
    answer_a, price_a = df_a.iloc[0].answer, df_a.iloc[0].credit_spent
    answer_b, price_b = df_b.iloc[0].answer, df_b.iloc[0].credit_spent
    return (answer_a, price_a), (answer_b, price_b)

In [212]:
shared_qs[0]

'What are the potential consequences of using LLMs in high-stakes applications without proper interpretability?'

In [213]:
get_answer_pair_for_budgets(normal_df, shared_qs[0], 5, 200)

(("The potential consequences of using LLMs in high-stakes applications without proper interpretability include the risk of perpetuating biases and discrimination, the potential for factual mistakes or outdated information, and the reliance on few-shot prompting and frozen language model parameters, which can limit the model's ability to adapt to new information, leading to potential errors or biases in high-stakes applications. For instance, LLMs may still make factual mistakes or contain outdated information, which can be detrimental in applications such as healthcare or finance. Moreover, LLMs can perpetuate biases present in the data they are trained on, which can result in unfair outcomes, particularly in areas like hiring or loan decisions. Additionally, the lack of interpretability in LLMs' decision-making processes can make it challenging to identify and address biases, further contributing to discrimination. Finally, the reliance on few-shot prompting and the freezing of all l

In [175]:
question = shared_qs[0]
(answer_a, price_a), (answer_b, price_b) = get_answer_pair(question)

/var/folders/f1/8sn11n6j4nn19h4w2dflk0ch0000gq/T/ipykernel_4414/3601656701.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["price_diff"] = df["credit_spent"].apply(lambda c: abs(c - price))
/var/folders/f1/8sn11n6j4nn19h4w2dflk0ch0000gq/T/ipykernel_4414/3601656701.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["price_diff"] = df["credit_spent"].apply(lambda c: abs(c - price))


In [262]:
def run_matches_normal_vs_noinspect(questions, budgets): 
    verdicts = []
    questions_and_budgets = product(questions, budgets)
    for question, budget in tqdm(questions_and_budgets):
        (answer_a, price_a), (answer_b, price_b) = get_answer_pair(question, budget=budget)

        print("*" * 50)
        print("*" * 50)
        print()
        print(question)
        print("-" * 50)
        print(f"Answer A (${price_a:.2f}):")
        print(answer_a)
        print("-" * 50)
        print(f"Answer B (${price_b:.2f}):")
        print(answer_b)

        print("=" * 50)
        print()

        verdict, debate, gold_answer = compare_answers(question, answer_a, answer_b, True)

        print("-" * 50)
        print("Gold answer:")
        print(gold_answer)

        print("-" * 50)
        print("Debate:")
        print(debate)

        verdicts.append(
            dict(
                question=question, 
                budget=budget,
                debate=debate, 
                gold_answer=gold_answer,
                verdict=verdict,
                answer_a=answer_a, 
                answer_b=answer_b,
            )
        )

        print()
        print("*" * 50)
        print("*" * 50)
        print()
    return verdicts


In [222]:
def run_matches_between_budget_pairs(questions, df, budget_pairs): 
    verdicts = []
    questions_and_budget_pairs = product(questions, budget_pairs)
    for question, budget_pair in tqdm(questions_and_budget_pairs):
        budget_a, budget_b = budget_pair
        (answer_a, price_a), (answer_b, price_b) = get_answer_pair_for_budgets(
            df, question, budget_a=budget_a, budget_b=budget_b
        )

        print("*" * 50)
        print("*" * 50)
        print()
        print(question)
        print("-" * 50)
        print(f"Answer A (${price_a:.2f}):")
        print(answer_a)
        print("-" * 50)
        print(f"Answer B (${price_b:.2f}):")
        print(answer_b)

        print("=" * 50)
        print()

        verdict, debate, gold_answer = compare_answers(question, answer_a, answer_b, True)

        print("-" * 50)
        print("Gold answer:")
        print(gold_answer)

        print("-" * 50)
        print("Debate:")
        print(debate)

        verdicts.append(
            dict(
                question=question, 
                budget_pair=budget_pair,
                debate=debate, 
                gold_answer=gold_answer,
                verdict=verdict,
            )
        )

        print()
        print("*" * 50)
        print("*" * 50)
        print()
    return verdicts


In [220]:
# verdicts = run_matches_normal_vs_noinspect(shared_qs[0:1], [5, 200])

In [258]:
verdicts_bpairs = run_matches_between_budget_pairs(shared_qs, normal_df, [[50, 200]])

0it [00:00, ?it/s]

**************************************************
**************************************************

What are the potential consequences of using LLMs in high-stakes applications without proper interpretability?
--------------------------------------------------
Answer A ($6.60):
The potential consequences of using LLMs in high-stakes applications without proper interpretability include the risk of errors, biases, and failures due to the lack of understanding of the decision-making process. In dynamic environments, the delay in response time and open-loop execution can lead to significant consequences, such as robotic failures or errors. Moreover, relying on LLMs without proper interpretability can make it challenging to address and correct errors, leading to potential problems in high-stakes applications.
--------------------------------------------------
Answer B ($155.32):
The potential consequences of using LLMs in high-stakes applications without proper interpretability are sign

1it [00:34, 34.23s/it]

--------------------------------------------------
Gold answer:
LLMs, or Large Language Models, are powerful tools that can generate human-like text based on the input they receive. However, using them in high-stakes applications without proper interpretability can lead to several potential consequences:

1. Misinformation: If the LLM is not properly trained or interpreted, it could generate incorrect or misleading information. This could have serious consequences in high-stakes applications like healthcare, finance, or legal advice.

2. Bias: LLMs learn from the data they are trained on. If this data contains biases, the LLM can perpetuate and even amplify these biases. This could lead to unfair or discriminatory outcomes.

3. Lack of Transparency: Without proper interpretability, it can be difficult to understand why an LLM is making certain decisions or predictions. This lack of transparency can make it hard to trust the LLM's outputs, especially in high-stakes situations.

4. Legal

2it [01:18, 40.34s/it]

...

--------------------------------------------------
Gold answer:
As an AI, I don't have real-time data or updates. However, as of my last update, there is a growing body of research comparing the performances of different language models on benchmarks that measure toxic language harms. The results can vary significantly depending on the specific language model and the benchmark used.

For instance, some studies have found that larger language models like GPT-3 can sometimes generate harmful or toxic outputs, even when they're not explicitly prompted to do so. Other models, like BERT, have been found to sometimes struggle with understanding the context of potentially harmful language, which can lead to false positives or negatives.

However, it's important to note that these models are continually being updated and improved. Many AI developers are working on ways to reduce the potential for harmful or toxic language generation, such as by implementing better moderation tools or by traini

19it [09:56, 30.28s/it]

--------------------------------------------------
Gold answer:
Continuous prompts and discrete prompts are terms often used in the field of machine learning, particularly in reinforcement learning.

Continuous prompts refer to a type of input or action space where the possible values are in a continuous range. For example, adjusting the temperature of a thermostat can be a continuous prompt, as the temperature can be set to any value within a certain range, like 18.5 degrees or 22.7 degrees.

On the other hand, discrete prompts refer to a type of input or action space where the possible values are distinct and separate. For example, choosing a channel on a television can be a discrete prompt, as you can only choose from a set number of channels, like channel 1, channel 2, channel 3, etc.

In the context of AI training, continuous prompts might require more complex algorithms to handle the infinite possibilities, while discrete prompts can be easier to manage due to their limited and d

20it [10:24, 31.21s/it]

--------------------------------------------------
Gold answer:
The architecture that has dominated natural language processing (NLP) benchmarks in recent years is called Transformer architecture. Models based on this architecture, such as BERT (Bidirectional Encoder Representations from Transformers), GPT-3 (Generative Pretrained Transformer 3), and T5 (Text-to-Text Transfer Transformer), have achieved state-of-the-art results on various NLP tasks.
--------------------------------------------------
Debate:
Michael: I think student A's answer is more comprehensive. They not only mentioned the transformer-based architecture but also explained why it has dominated NLP benchmarks. They discussed the architecture's ability to handle long-range dependencies, its parallelization capabilities, and its ability to handle out-of-vocabulary words. 

Bobby: I agree that student A's answer is detailed, but it's also quite verbose and goes off on tangents. They mentioned parallelization capabilities

In [259]:
len([1 for v in verdicts_bpairs if v["verdict"] == "B"])

13

In [261]:
# dump_dict(verdicts_bpairs, "./assets/verdicts_gpt4_fup-v3_budgets-50-200.json")

In [291]:
verdicts_mpairs = run_matches_normal_vs_noinspect(shared_qs, [25])

1it [00:00,  2.16it/s]

**************************************************
**************************************************

What are the potential consequences of using LLMs in high-stakes applications without proper interpretability?
--------------------------------------------------
Answer A ($24.88):
The potential consequences of using LLMs in high-stakes applications without proper interpretability include a lack of transparency and explainability, potential bias and discrimination, and security risks. However, the extent to which these consequences can be generalized to all high-stakes applications that use LLMs is unclear and requires further investigation.

To address the concerns of bias and discrimination, it is essential to ensure that the data used to train LLMs is diverse and representative of all groups of people, and that the models are designed with fairness and equity in mind. Additionally, implementing fairness-aware regularization techniques, such as adversarial training, can help reduce 

7it [00:00, 12.25it/s]

--------------------------------------------------
Gold answer:
In-context learning can occur even when there's a distribution mismatch between prompts and pretraining data. However, the effectiveness of the learning might be impacted. The AI model might struggle to generalize or make accurate predictions if the prompts it's receiving during fine-tuning are significantly different from the data it was pretrained on. This is because the model learns patterns and structures from the pretraining data, and applies this knowledge to the prompts it receives later. If there's a large mismatch, the model might not have learned the necessary patterns to handle the new prompts effectively.
--------------------------------------------------
Debate:
Michael: I think student A's answer is better. It directly addresses the question and provides a detailed explanation of how in-context learning can occur even when there's a distribution mismatch between prompts and pretraining data. The student also 

10it [00:00, 15.26it/s]

...

--------------------------------------------------
Gold answer:
As an AI, I don't have real-time data or updates. However, as of my last update, there is a growing body of research comparing the performances of different language models on benchmarks that measure toxic language harms. The results can vary significantly depending on the specific language model and the benchmark used.

For instance, some studies have found that larger language models like GPT-3 can sometimes generate harmful or toxic outputs, even when they're not explicitly prompted to do so. Other models, like BERT, have been found to sometimes struggle with understanding the context of potentially harmful language, which can lead to false positives or negatives.

However, it's important to note that these models are continually being updated and improved. Many AI developers are working on ways to reduce the potential for harmful or toxic language generation, such as by implementing better moderation tools or by traini

19it [05:24, 32.52s/it]

--------------------------------------------------
Gold answer:
Continuous prompts and discrete prompts are terms often used in the field of machine learning, particularly in reinforcement learning.

Continuous prompts refer to a type of input or action space where the possible values are in a continuous range. For example, adjusting the temperature of a thermostat can be a continuous prompt, as the temperature can be set to any value within a certain range, like 18.5 degrees or 22.7 degrees.

On the other hand, discrete prompts refer to a type of input or action space where the possible values are distinct and separate. For example, choosing a channel on a television can be a discrete prompt, as you can only choose from a set number of channels, like channel 1, channel 2, channel 3, etc.

In the context of AI training, continuous prompts might require more complex algorithms to handle the infinite possibilities, while discrete prompts can be easier to manage due to their limited and d

20it [05:42, 17.11s/it]

--------------------------------------------------
Gold answer:
The architecture that has dominated natural language processing (NLP) benchmarks in recent years is called Transformer architecture. Models based on this architecture, such as BERT (Bidirectional Encoder Representations from Transformers), GPT-3 (Generative Pretrained Transformer 3), and T5 (Text-to-Text Transfer Transformer), have achieved state-of-the-art results on various NLP tasks.
--------------------------------------------------
Debate:
Michael: Well, this is a no-brainer. Student A's answer is not even an answer. It's just 'nan'. 

Bobby: I agree, Michael. Student A's answer doesn't provide any information at all. 

Michael: On the other hand, Student B's answer is quite informative. They mentioned the GPT series and BERT, which are indeed models based on the Transformer architecture.

Bobby: Yes, but they didn't mention the Transformer architecture itself, which is the core of the question. 

Michael: That's true

In [292]:
len([1 for v in verdicts_mpairs if v["verdict"] == "B"])

15

In [295]:
# _ = dump_dict(verdicts_mpairs, "./assets/verdicts_gpt4_fup-v3-vs-noinspect_budget-25.json")

In [270]:
noinspect_df.budget.unique()

array([ 25.00310457,  10.00414993,  49.99884974,  99.98298285,
         4.99781092, 199.93653676])

In [271]:
normal_df.budget.unique()

array([ 10.00414993, 199.93653676,  99.98298285,  25.00310457,
        49.99884974,   4.99781092])